In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import datetime
import torch
from torch.nn import functional as F
import scipy.stats
import sklearn.datasets
import glob

import sys
sys.path.append("../")
import curvvae_lib.train.predictive_passthrough_trainer as ppttrainer
import curvvae_lib.architecture.passthrough_vae as ptvae
import curvvae_lib.architecture.load_model as lm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-5z0wi7s1 because the default path (/home/tsr42/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
/home/tsr42/anaconda3/envs/curvvaecuda/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def plot_closure():
    plt.show()
    #plt.close()

# Dataset

In [3]:
foodname = "banana"
foldername = f"fork_trajectory_{foodname}"
savefilename = f"{foodname}_clean_pickups"

In [4]:
train = []
training_ts = np.linspace(0,1,64)
attempt = 1
while True:
    try:
        raw_vals = np.load(f"{savefilename}/pickup_attempt{attempt}.npy")
        train.append(raw_vals.T.flatten())
    except:
        print(f"We found {attempt-1} pickup attempts")
        break
    attempt += 1

train = np.array(train).reshape(-1,7,64)
all_points = train[:,:,:]

time_shape = list(all_points.shape)
time_shape[1] = 1
# why be smart when you can be dumb
t = np.ones(time_shape)
for i in range(time_shape[2]):
    t[:,:,i] = t[:,:,i] * i / (time_shape[2] + 0.0)

all_points = np.concatenate((t, all_points), axis=1)
all_points = all_points.transpose(0,2,1)
print(all_points.shape)
all_poses = all_points.reshape(-1,8)

# See http://localhost:8889/notebooks/scratchwork/2021-09-17%20Rotation%20Scaling.ipynb
# for why we want quaternion values to be multiplied by 0.16 when position values are in meters 
# (if the relevant distance scale of the fork is 0.08 meters, ie: 8cm).
# scaling term doesn't affect time, so don't use time in calculation

mean = np.mean(all_poses, axis=0)
mean[0] = 0 # don't scale time
variance = np.var(all_poses[:,1:], axis=0) # don't scale time
print(mean)
print(variance)
position_std = np.sqrt(np.max(variance))
print("std of: ", position_std)
position_scaling = 1/position_std
rotation_scaling = 0.16 * position_scaling

We found 155 pickup attempts
(155, 64, 8)
[ 0.          0.00227658 -0.00117697  0.0121782  -0.04935249  0.37343377
 -0.89429268 -0.01921521]
[2.63014114e-05 3.40430938e-05 1.00819967e-04 7.90561700e-03
 3.18947674e-02 7.03375426e-03 1.11414372e-02]
std of:  0.17859106200728153


In [5]:
def print_to_csv(mean):
    string = ""
    for i,m in enumerate(mean):
        string += "%0.8f" % m
        if i < len(mean)-1:
            string+= ","
    return string
print(f"\
      mean = np.array(({print_to_csv(mean[1:])}))\n\
      ps = {position_scaling}\n\
      rs = {rotation_scaling}\n")
def scale_dataset(input_points):
    points = input_points - mean
    poss = position_scaling
    rts = rotation_scaling
    points = (points * np.array((1,poss,poss,poss,rts,rts,rts,rts)))
    return points

def unscale_poses(input_points):
    poss = position_scaling
    rts = rotation_scaling
    points = (input_points / np.array((poss,poss,poss,rts,rts,rts,rts)))
    points = points + mean[1:]
    return points 
    
def unscale_dataset(input_points):
    poss = position_scaling
    rts = rotation_scaling
    points = (input_points / np.array((1,poss,poss,poss,rts,rts,rts,rts)))
    points = points + mean
    return points

      mean = np.array((0.00227658,-0.00117697,0.01217820,-0.04935249,0.37343377,-0.89429268,-0.01921521))
      ps = 5.599384363139225
      rs = 0.895901498102276



In [6]:
dataset = scale_dataset(all_points)
t_all_points = torch.tensor(dataset,dtype=torch.float32)

## 2D VAE Fit to Dataset

In [7]:
def LoadDataBatch(all_points, batchsize, passthroughdim, predictive, device):
    """Sample Pair of Points from Trajectory"""
    # all_points should be of dimension: num_trajectories, numtimesteps, 1+spatialdims
    traj_ids = np.random.choice(all_points.shape[0], batchsize)
    t1_ids = np.random.choice(all_points.shape[1], batchsize)
    if predictive:
        t2_ids = np.random.choice(all_points.shape[1], batchsize)
    else:
        t2_ids = t1_ids
    return (torch.tensor(all_points[traj_ids, t1_ids,passthroughdim:], dtype=torch.float).to(device),
            torch.tensor(all_points[traj_ids, t1_ids,:passthroughdim], dtype=torch.float).to(device),
            torch.tensor(all_points[traj_ids, t2_ids,passthroughdim:], dtype=torch.float).to(device),
            torch.tensor(all_points[traj_ids, t2_ids,:passthroughdim], dtype=torch.float).to(device))

In [8]:
class Loader(object):
    def __init__(self, all_points, batchsize, passthroughdim, epochnumbatches, predictive, device):
        self.all_points = all_points
        self.batchsize = batchsize
        self.passthroughdim = passthroughdim
        self.epochnumbatches = epochnumbatches
        self.predictive = predictive
        self.device = device
        
    def __iter__(self):
        self.n = 0
        return self
    
    def __next__(self):
        if self.n >= self.epochnumbatches:
            # https://docs.python.org/3/library/exceptions.html#StopIteration
            raise StopIteration
        self.n += 1
        return LoadDataBatch(self.all_points, self.batchsize, self.passthroughdim, self.predictive, self.device)
    

device = "cuda"
train_loader = Loader(dataset[:,np.newaxis,:], 256, 0,10,predictive=False, device=device)
  

In [9]:
dataset_dim = dataset.shape[-1]
def make_vae(latent_dim):
    input_dim = dataset_dim 
    passthrough_dim = 0
    emb_layer_widths = [1000]
    recon_layer_widths = [1000]
    dtype = torch.float
    model = ptvae.FCPassthroughVAE(input_dim, passthrough_dim, latent_dim,
        emb_layer_widths, recon_layer_widths, dtype)
    return model

In [10]:
testname = f"trainedmodels/{foodname}_"

In [11]:
all_models = ['trainedmodels/banana_lat3_curvreg0_beta0.001_20220209-144826']
#all_models = [f'trainedmodels/banana_lat3_curvreg0.001_beta0.001_20220209-120436']
all_models = [f'trainedmodels/banana_lat3_curvreg0_beta0.01_20220209-152954']
all_models

['trainedmodels/banana_lat3_curvreg0_beta0.01_20220209-152954']

In [12]:
latent_lim_values = np.array((2.,2,2))

In [13]:
modelname = all_models[0]
loaded_vae = lm.load_model(modelname)


In [14]:
import transforms3d as t3d

In [15]:
def loss_function(numpy_latent,target_angle, verbose=False):
    targ_ang_rads = 0.4 - (target_angle / 180. * np.pi)
    #print(targ_ang_rads)
    latent = torch.tensor(numpy_latent,dtype=torch.float32).reshape(1,3)
    t = torch.tensor((0),dtype=torch.float32).reshape(1,1)
    scaled_pose, _ = loaded_vae.decode(latent,t)
    pose = unscale_poses(scaled_pose.detach().cpu().numpy())
    quat = pose[0,3:]
    base_quat = (0,0,1,0)
    quat = t3d.quaternions.qmult(base_quat,quat)
    eul = np.array(t3d.euler.quat2euler(quat,axes='sxyz'))
    if verbose:
        print(eul * 180/np.pi)
    return np.sum(np.square(eul - (targ_ang_rads,0,-0.79478576)) * np.array((1,0.0001,0.0001))) # ok for a little side to side, but gotta be correct tilt

In [16]:
# https://pyswarms.readthedocs.io/en/latest/examples/usecases/train_neural_network.html#Constructing-a-custom-objective-function
def loss_function_batch(x, target_angle):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [loss_function(x[i],target_angle) for i in range(n_particles)]
    return np.array(j)

In [17]:
import pyswarms as ps

final_answers = []
for target_ang in [30,45,60]:
    lim = 3
    curbest = None
    while curbest is None:# or (np.sqrt(curbest[0])/np.pi) * 180 > 1: # keep expanding the search radius until within 1 degree of desired angle
        #lim += 0.5
        for _ in range(10):
            # Set-up hyperparameters
            options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

            # Call instance of GlobalBestPSO
            optimizer = ps.single.GlobalBestPSO(n_particles=30, dimensions=3,
                                                options=options, bounds=((-lim,-lim,-lim),(lim,lim,lim)))

            # Perform optimization
            stats = optimizer.optimize(loss_function_batch, target_angle=target_ang, iters=200)

            if curbest is None or stats[0] < curbest[0]:
                print("updating best")
                curbest = stats
    final_answers.append(curbest)
print(final_answers)

2022-07-25 19:16:15,296 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|200/200, best_cost=4.45e-5
2022-07-25 19:16:16,950 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 4.4506714707206825e-05, best pos: [-1.24974167 -0.0766631   2.0429148 ]
2022-07-25 19:16:16,958 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


updating best


pyswarms.single.global_best: 100%|██████████|200/200, best_cost=4.39e-5
2022-07-25 19:16:18,538 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 4.3915751041641925e-05, best pos: [-0.72739369 -2.85220755  2.09472872]
2022-07-25 19:16:18,545 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


updating best


pyswarms.single.global_best: 100%|██████████|200/200, best_cost=4.45e-5
2022-07-25 19:16:20,115 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 4.4502787075631383e-05, best pos: [-1.20350395 -0.09042027  2.04351418]
2022-07-25 19:16:20,121 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|200/200, best_cost=4.39e-5
2022-07-25 19:16:21,538 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 4.389041152546449e-05, best pos: [-1.5804171  -2.98966789  2.09215759]
2022-07-25 19:16:21,545 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


...

updating best


pyswarms.single.global_best: 100%|██████████|200/200, best_cost=0.186
2022-07-25 19:16:50,625 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.1864316322231737, best pos: [-1.39812148 -0.92936399  2.9132612 ]
2022-07-25 19:16:50,633 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|200/200, best_cost=0.186
2022-07-25 19:16:52,160 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.1864092913351347, best pos: [-1.30906081 -0.95564429  2.9157194 ]
2022-07-25 19:16:52,168 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|200/200, best_cost=0.188
2022-07-25 19:16:53,642 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.18812293262490443, best pos: [-0.22981294 -0.44345184  2.89790663]
2022-07-25 19:16:53,647 - pyswarms.single.g

updating best


pyswarms.single.global_best: 100%|██████████|200/200, best_cost=0.187
2022-07-25 19:16:59,639 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.18730026849695386, best pos: [-0.76469811 -0.6530216   2.90367867]
2022-07-25 19:16:59,646 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|200/200, best_cost=0.187
2022-07-25 19:17:01,104 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.1869749809118967, best pos: [-1.10697351 -0.713414    2.88924161]


[(4.3422275479014246e-05, array([ 2.64505778, -2.24123117,  2.09124194])), (0.02888499045942051, array([-2.40510984, -0.98495732,  2.93280035])), (0.18624782970494155, array([-2.20007772, -1.07386418,  2.94487741]))]


In [18]:
for fb in final_answers:
    print(np.sqrt(fb[0]) / np.pi * 180)

0.3775538659244098
9.737752825997088
24.726818599069475


In [19]:
for fb in final_answers:
    print(f"[{','.join([f'{s}' for s in fb[1]])}],")

[2.6450577835825406,-2.2412311691819014,2.0912419381190945],
[-2.405109843025896,-0.984957320738932,2.9328003540472896],
[-2.2000777160967875,-1.0738641815565706,2.9448774111432066],


In [20]:
for fb in final_answers:
    loss_function(fb[1],1000,True)


[-7.06956376 -5.60412063 -8.22040687]
[-12.35482768 -10.35527469  -0.67235714]
[-12.35915129 -10.42374576  -0.72029321]


In [21]:
0.4*180/np.pi

22.91831180523293

In [22]:
"""
[0.2478165554759397,-1.249154366817157,-1.5198855746754794],
[0.10129095212372306,-2.8613022436706315,0.9409312592318927],
[0.5889920784970968,-2.3291049154306642,2.6950374180789747],
"""

'\n[0.2478165554759397,-1.249154366817157,-1.5198855746754794],\n[0.10129095212372306,-2.8613022436706315,0.9409312592318927],\n[0.5889920784970968,-2.3291049154306642,2.6950374180789747],\n'